# Imports

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

import math
from collections import Counter

# Import em download
#import decision_tree_matott as tree
#from decision_tree_matott.tree import ID3
#from decision_tree_matott.tree import C45
#from decision_tree_matott.tree import CART

# Import local
import tree

# Preparação dos dados

## Base de dados

In [2]:
# Leitura e foramtação dos dados
dados = pd.read_csv('Data/titanic.csv', sep=';', na_values=["", " ", "NA", "N/A"])

In [3]:
dados = dados['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'].str.split(',', expand=True)
dados.columns = ['PassengerId', 'Survived', 'Pclass', 'Surname', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

In [4]:
dados_base = pd.DataFrame(dados)

In [5]:
dados_base

,PassengerId,Survived,Pclass,Surname,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"""Braund","Mr. Owen Harris""",male,22,1,0,A/5 21171,7.25,,S
1,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"""Heikkinen","Miss. Laina""",female,26,0,0,STON/O2. 3101282,7.925,,S
3,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"""Allen","Mr. William Henry""",male,35,0,0,373450,8.05,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"""Montvila","Rev. Juozas""",male,27,0,0,211536,13,,S
887,888,1,1,"""Graham","Miss. Margaret Edith""",female,19,0,0,112053,30,B42,S
888,889,0,3,"""Johnston","Miss. Catherine Helen """"Carrie""""""",female,,1,2,W./C. 6607,23.45,,S
889,890,1,1,"""Behr","Mr. Karl Howell""",male,26,0,0,111369,30,C148,C


## Atributos irrelevantes

In [6]:
# PassengerId é condizente com a própria numeração na tabela e único a cada um
dados_base = dados_base.drop(columns=['PassengerId'])
# Nome é um atributo sempre único e sem impacto na sobrevivência do passageiro
dados_base = dados_base.drop(columns=['Surname', 'Name'])
# Ticket é provavelmente irrelevante (supondo que não reflete fatores como localização no navio e outros)
dados_base = dados_base.drop(columns=['Ticket'])
# Cabine pode ser relevante, mas tem uma taxa de 77% dos dados nulos, sendo então impreciso para a análise
dados_base = dados_base.drop(columns=['Cabin'])

## Tratamento de dados nulos

In [7]:
# Obter moda e inserir no lugar
for column in dados_base.columns:
    grupo = dados_base[column]

    # Determinar moda
    moda = Counter(grupo).most_common(1)[0][0]
    # Para evitar se moda for valor nulo
    if not moda:
        moda = Counter(grupo).most_common(2)[1][0]

    # Preencher dados vazios
    dados_base[column] = dados_base[column].fillna(moda)
    for linha in range(len(dados_base)):
        if ( type(dados_base[column][linha]) == str ):
            if not ( len(dados_base[column][linha]) > 0 ):
                dados_base.loc[linha, column] = ( moda )
        elif ( type(dados_base[column][linha]) == int ):
            if not ( dados_base[column][linha] >= 0 ):
                dados_base.loc[linha, column] = ( moda )
        elif ( dados_base[column][linha] == None ):
            dados_base.loc[linha, column] = ( moda )

## Separação entre dados para cada algoritmo

In [8]:
# ID3
dados_id3 = dados_base.copy()
# C4.5
dados_c45 = dados_base.copy()
# CART
dados_cart = dados_base.copy()

## Tratamento de dados contínuos

In [9]:
colunas_continuas = ['Age', 'Fare']
n_intervalos = 10
# ID3
# Obter intervalo dos valores e determinar a qual parte do intervalo o valor pertence
for coluna in colunas_continuas:
    min = float(dados_id3[coluna][0])
    max = float(dados_id3[coluna][0])
    for instancia in range(len(dados_id3[coluna])):
        dados_id3.loc[instancia, coluna] = float(dados_id3[coluna][instancia])
        if dados_id3[coluna][instancia] < min:
            min = dados_id3[coluna][instancia]
        elif dados_id3[coluna][instancia] > max:
            max = dados_id3[coluna][instancia]
    interval_range = (max-min)/n_intervalos
    for instancia in range(len(dados_id3[coluna])):
        dados_id3.loc[instancia, coluna] = math.ceil(dados_id3[coluna][instancia]/interval_range)
# C4.5 e CART não precisam de tratamento de dados contínuos, mas é bom garantir que eles
#  serão tratados como valores numéricos
for coluna in colunas_continuas:
    for instancia in range(len(dados_c45[coluna])):
        dados_c45.loc[instancia, coluna] = float(dados_c45[coluna][instancia])
    for instancia in range(len(dados_cart[coluna])):
        dados_cart.loc[instancia, coluna] = float(dados_cart[coluna][instancia])

# Partição treino/teste

In [10]:
# Separação de resultados
result_column = 'Survived'
# ID3
dados_id3_atributos  = dados_id3.drop(columns=[result_column])  # Atributos
dados_id3_resultados = dados_id3[result_column]                 # Conclusão
# C4.5
dados_c45_atributos  = dados_c45.drop(columns=[result_column])  # Atributos
dados_c45_resultados = dados_c45[result_column]                 # Conclusão
# CART
dados_cart_atributos  = dados_cart.drop(columns=[result_column])  # Atributos
dados_cart_resultados = dados_cart[result_column]                 # Conclusão

# Separar dados em treino e teste
test_sample = 0.20
# ID3
dados_id3_atributos_treino, dados_id3_atributos_teste, dados_id3_resultados_treino, dados_id3_resultados_teste = train_test_split(dados_id3_atributos, dados_id3_resultados, test_size = test_sample, random_state = 42)
# C4.5
dados_c45_atributos_treino, dados_c45_atributos_teste, dados_c45_resultados_treino, dados_c45_resultados_teste = train_test_split(dados_c45_atributos, dados_c45_resultados, test_size = test_sample, random_state = 42)
# CART
dados_cart_atributos_treino, dados_cart_atributos_teste, dados_cart_resultados_treino, dados_cart_resultados_teste = train_test_split(dados_cart_atributos, dados_cart_resultados, test_size = test_sample, random_state = 42)

# Montagem das árvores

# Restrições

In [11]:
altura_max = 10
min_informatio_gain = 0.150
min_gain_ratio      = 0.100
min_gini            = 0.100

# Montagem

In [ ]:
# A chamada das classes muda se o import de classe for local (prefixo de 'tree.')
# ID3
tree_id3 = tree.ID3(max_height= altura_max, min_information= min_informatio_gain, results= dados_id3_resultados_treino)
tree_id3.create_tree(data= dados_id3_atributos_treino, result= dados_id3_resultados_treino)
# C4.5
tree_c45 = tree.C45(max_height= altura_max, min_information= min_gain_ratio, results= dados_c45_resultados_treino)
tree_c45.create_tree(data= dados_c45_atributos_treino, result= dados_c45_resultados_treino)
# CART
tree_cart = tree.CART(max_height= altura_max, min_information= min_gini, results= dados_cart_resultados_treino)
tree_cart.create_tree(data= dados_cart_atributos_treino, result= dados_cart_resultados_treino)

# Resultados

## ID3

### Árvore

In [13]:
print(tree_id3.root)

0 - 0 (/'1' 268//'0' 444/) | Informacao: 0.21537571560753244 | Regra derivada: Sex


In [14]:
print(tree_id3.__str__())

{}  0 - 0 (/'1' 268//'0' 444/) | Informacao: 0.21537571560753244 | Regra derivada: Sex

  {female}  1 - 1 (/'1' 181//'0' 64/) | Informacao: 0.23487005404128802 | Regra derivada: Pclass





### Teste

## C4.5

### Árvore

In [15]:
print(tree_c45.root)

0 - 0 (/1 268//0 444/) | Razão de ganho: 0.1159568277286427 | Regra derivada: Sex


In [16]:
print(tree_c45.__str__())

{}  0 - 0 (/1 268//0 444/) | Razão de ganho: 0.1159568277286427 | Regra derivada: Sex




### Teste

## CART

### Árvore

In [17]:
print(tree_cart.__str__())

{24.0}  0 - 0 (/1 268//0 444/) | Razão de ganho: 0.499222913257106 | Regra derivada: Age
  {>34.5}  1 - 0 (/1 139//0 213/) | Razão de ganho: 0.4975585849188431 | Regra derivada: Age
    {>44.0}  2 - 0 (/1 74//0 108/) | Razão de ganho: 0.4976777974775972 | Regra derivada: Age
      {>51.0}  3 - 0 (/1 35//0 58/) | Razão de ganho: 0.49598489681630775 | Regra derivada: Age
        {>57.0}  4 - 0 (/1 17//0 27/) | Razão de ganho: 0.48832422650185037 | Regra derivada: Age
          {>61.0}  5 - 0 (/1 9//0 15/) | Razão de ganho: 0.4958024691358024 | Regra derivada: Age
            {>1}  6 - 0 (/1 4//0 8/) | Razão de ganho: 0.4635416666666667 | Regra derivada: Pclass
              {>2}  7 - 0 (/1 2//0 3/) | Razão de ganho: 0.4388888888888889 | Regra derivada: Pclass


              {<=2}  7 - 0 (/1 2//0 3/) | Razão de ganho: 0.4388888888888889 | Regra derivada: Pclass


            {<=59.0}  6 - 0 (/1 5//0 7/) | Razão de ganho: 0.48489795918367345 | Regra derivada: Age
              {>1}  7 - 0

### Teste